In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import compare_ssim as ssim

def mse (image1, image2):
    error = np.sum((image1.astype('float') - image2.astype('float'))**2)
    #Smaller is more similar
    error = error/float(image1.shape[0]*image1.shape[1])
    return error

def dssimCompare (image1, image2):
    #Smaller is more similar
    return (1 - ssim(image1, image2, 3))/2

def psnrCompare (image1, image2):
    #See http://docs.opencv.org/2.4/doc/tutorials/highgui/video-input-psnr-ssim/video-input-psnr-ssim.html#videoinputpsnrmssim
    #Larger is more similar!
    error = 10*log10((255*255)/mse(image1, image2))
    return error

def msssimCompare(image1, image2):
    return error
    
def plotError(frameCount):
    print "Now plotting errors"
    #Create a list of our errors between frame t and t-1 and between frame t and t+1 for all frames
    #list comp then a for loop to populate
    frameErrors =  [[0 for x in range(2)] for y in range(frameCount)]
    for i in range(1, frameCount - 1):
        image1 = cv2.imread("frame%0*d.png" %(4,i))
        image2 = cv2.imread("frame%0*d.png" %(4,i-1))
        frameErrors[i-1][0] = dssimCompare(image1, image2)
        image2 = cv2.imread("frame%0*d.png" %(4,i+1))
        frameErrors[i-1][1] = dssimCompare(image1, image2)
    #find a way to save our array with numpy here
    
    #declare heat dist plot and assign axis
    #Maybe we can make the range(1, frameCount-1) a term on it's own to avoid the duplication we have above and below
    x = range(1, frameCount-1)
    y = range(1, frameCount-1)
    z = #these are the actual values from our array
    X, Y = np.meshgrid(x, y)
    Z = z.reshape(len (x), len (y))
    plt.pcolormesh(X, Y, Z)
    plt.title('DSSIM Value Between Frames')
    plt.grid(True)
    plt.savefig("similarityplot.png")
    plt.show()

#Main Function
plotError()
